# Generics
---

If you notice, we have two problems with the way we declared `IntList` and its operations,
1. It's restricted to `Int` elements, which means we would need to re-declare a linked list for each type of elements we want to hold.
2. The operations defined on it, some of them follow a common pattern. Lot of code repetition.

To mitigate the first problem, we need some mechanism to _abstract over types_ so that we can use same ADT with different concrete types. And for the later problem, we need a way to _abstract over metods_ so that we can avoid repetition of common patterns.

In this notebook, we are going to focus on the facility called `Generics` to solve the first problem. Let's see the problem in practice.

In [1]:
sealed trait IntList
final case object IntEnd extends IntList
final case class IntPair(head: Int, tail: IntList) extends IntList

defined trait IntList
defined object IntEnd
defined class IntPair

In order to hold elements of type `String` or any user-defined type, say `Programmer`, we obviously can't re-use the `IntList`. We may need to create another list structure to hold values for those specific types. You will end up with hundred linked list ADTs for hundred types.

In [2]:
sealed trait StringList
final case object StringEnd extends StringList
final case class StringPair(head: String, tail: StringList) extends StringList

defined trait StringList
defined object StringEnd
defined class StringPair

In [3]:
case class Programmer(name: String, lang: String)

sealed trait ProgrammerList
final case object ProgrammerEnd extends ProgrammerList
final case class ProgrammerPair(head: Programmer, tail: ProgrammerList) extends ProgrammerList

defined class Programmer
defined trait ProgrammerList
defined object ProgrammerEnd
defined class ProgrammerPair

Because Scala has `Any` type, which can hold values of any type, one can define a linked list to hold "any" value but with that we lose the type-safety. Though we have the '.asInstanceOf' method to cast a value of type `Any` to any desired type but that too is not type-safe and could lead to various bugs at runtime. We need a proper solution so that we can re-use the ADT (or method too) without sacrificing the type-safety.

In [4]:
sealed trait AnyList
final case object AnyEnd extends AnyList
final case class AnyPair(head: Any, tail: AnyList) extends AnyList

val anyList: AnyList = AnyPair("Hello", AnyEnd)
val anyHead = anyList match {
    case AnyPair(head, _) => head
}
val stringHead = anyHead.asInstanceOf[String]

defined trait AnyList
defined object AnyEnd
defined class AnyPair
anyList: AnyList = AnyPair(head = "Hello", tail = AnyEnd)
anyHead: Any = "Hello"
stringHead: String = "Hello"

Notice the types of `anyHead` and `stringHead` values. When we extracted the "head" from `AnyList`, it was of the type `Any` and to use the value in any meaningful way we needed to cast the value to its appropriate type.

---

`Generics` allow us to declare types (ADT) and methods so that they can be used with objects of various types while providing compile-time type safety. Let's create simple data structure to hold single value of any type.

In [5]:
final case class Box[A](value: A)

val intBox = Box[Int](1)
val stringBox = Box("1") // type of value is inferred, expreesion is similar to Box[String]("1")

val int = intBox.value
val string = stringBox.value

defined class Box
intBox: Box[Int] = Box(value = 1)
stringBox: Box[String] = Box(value = "1")
int: Int = 1
string: String = "1"

#### couple of things happened in above cell:

1. we declared a generic `Box` type
    - generic type (or method) takes `type parameter`, in this case `A`. 
    - in squre bracket after the name of type (or method).
    - we can add any number of type parameter, and we can name it as we please.
        - `final case class Box[SomeType](value: SomeType)`
        - `final case class Pair[X, Y](a: X, b: Y)`
2. we created instances of box with type `Int` and `String`.
    - we can omit the type parameter while creating an instance of generic type, if it's inferrable by Scala.
        - `Pair[Int, Programmer](1, Programmer("A", "B"))`
3. we queried the value inside the box.
    - notice the type of value is preserved after querying the value
    
#### couple of things to keep in mind:

1. read `Box[Int]` generic type as, a box of integer. 
2. a generic type without type parameter is _not_ a type. The `Box` alone is not a type.
3. type parameter should be a type, which means `Box[Box]` is invalid becuase the second `Box` is not a type, as per (2).
4. fill in the type is same as replacing a value of `x` in some math equation,
    - for equation, `x^2 + 2x + 10` and where `x` is 2, expression will be `2^2 + 2*2 + 10`.
    - similarly, for `Box[A](value: A)` and where `A` is `String`, type will be `Box(value: String)`.
        - or, for `Pair[X, Y](a: X, b: Y)` where `X` is `Int` and `Y` is `Programmer`, type will be `Pair(a: Int, b: Programmer)`.
5. the code will not compile if the expression is`Box[Int]("a")`,
    - `Box[Int]` => `Box(value: Int)`
    - because we are passing a `String` ("a") where expected type is `Int`.

In [5]:
val intBox = Box[Int]("a")

cmd5.sc:1: type mismatch;
 found   : String("a")
 required: Int
val intBox = Box[Int]("a")
                      ^Compilation Failed

: 

Now that have some familiarity with `Generics`, let's re-define our linked list structure using type parameter.

In [6]:
sealed trait List[A]
final case class End[A]() extends List[A]
final case class Pair[A](head: A, tail: List[A]) extends List[A]

defined trait List
defined class End
defined class Pair

That's it. We now of a list of `A`, where `A` could be any typel. Notice that we have to change `End` from `object` to `class` becuase, <u>objects can't be type parameterized</u>, and we need to pass that `A` to the trait `List`. We can remove that `A` from `End` and change it back to `object` but we need to wait for yet another mechanism that Scala provides, Type Variance. Let's create a list of string, and let's see if we preserve the type of head on querying. Notice the type of `stringHead`, no need to use `.asInstanceOf`.

In [7]:
val stringList: List[String] = Pair[String]("Hello", End[String]())
val stringHead = stringList match {
    case Pair(head, _) => head
}

stringList: List[String] = Pair(head = "Hello", tail = End())
stringHead: String = "Hello"

#### methods with type parameter
---

in similar way, we can use type parameter to accept objects of different type in a method.

In [8]:
def identity[T](x: T): T = x

identity(1)
identity[String]("string")

defined function identity
res7_1: Int = 1
res7_2: String = "string"

The `identity` method accepts any type of object and return it as it is. Again, use the mental model of math equation.

For, `def identity[T](x: T): T = x` and where `T` is `Int`, it becomes `def identity(x: Int): Int = x`.

And it type safe too, `identity[String](1)` won't compile.

---

Let's try to define some operation on our generic list, say `product`.

In [8]:
object ListOps {
    
    def product[A](list: List[A]): Int = list match {
        case End() => 0
        case Pair(head, tail) => head * product(tail)
    }
    
}

cmd8.sc:5: value * is not a member of type parameter A
        case Pair(head, tail) => head * product(tail)
                                      ^Compilation Failed

: 

Woh, what just happened? The compiler is complaining that "value * is not a member of type parameter A". Okay, let's break it down. In Scala, each method (operations, such as `*`) belongs to a type. Here, `A` is placeholder for a type that user will provide while calling the `product` method, which means, at the time of declaration, the compiler (or we) don't know what the actual type `A` is. So, obviously compiler won't allow us to perform `*` operation on the unknown type `A`.

This is actually a good thing. It restricts the number of implementaions the `product` method can have, or any generic method for that matter. We can provide the information to compiler about what types are allowed to pass while calling the method, but that topic is for another notebook.

In short, for now, we won't be allowed to implement any operation in which we need to deal with a value of unknown type. But there are still some operations in which we don't need to use list's elements or need to know the type of its elements. Such operations are `length`, `concat`, etc... to calculate the length of list we don't need to know the type of element or we don't need to use element to perform any operation. Let's implement the `length` operation.

In [9]:
object ListOps {
    
    def length[A](list: List[A]): Int = list match {
        case End() => 0
        case Pair(_, tail) => 1 + length(tail)
    }
    
}

defined object ListOps

In [10]:
val list: List[String] = Pair("A", Pair("B", Pair("C", End[String]())))
ListOps.length(list) // 3

list: List[String] = Pair(
  head = "A",
  tail = Pair(head = "B", tail = Pair(head = "C", tail = End()))
)
res9_1: Int = 3

As you can see, we defined a list ["A", "B", "C"] and `length` computed the answer `3`, as expected. No need to know what type of values the list holds. 

Notice the implementation of `length` method, in `Pair` case, we perform `1 + length(tail)` computation. The reason compiler allows us to call `+` is,
1. it knows that `1` has type `Int`, becuase of literal expression.
2. it also knows that the type of `length(tail)` is `Int`, becuase `length` returns a value of type `Int`.

### conclusion
---

In this notebook, we went through limited features of generic. We defined data structure (and method) using type parameter(s), been able to write operations where we don't need to use element values. We still need to review `type variance` and `type constraint` to fully leverage the mechanism of `Generics`.

### Exercise
---

1. Re-define the `StringTree` (see `06-recursive-data` notebook) as `Tree` and use generics to create an instance of tree with any type of objects.

2. Define following method, `def divide[A, B](a: A, b: B): Int = a / b`. Guess the answer of `divide(4, 2)` expression before running it. Explain the evaluation, and errors (if any). 

3. Declare a data structure from following statement. [hint: you may need to use two type parameters]
    - a computation can hold a result of any type or it could fail and hold an error of any type
    
4. See the following snippet,
   ```
   final case class Single[V](value: V)
   final case class Tuple[A, B](a: A, b: Single[B])
   final case class Triple[SomeTypeA, T, HELLO](a: SomeTypeA, b: Single[T], c: Tuple[T, HELLO])
   ```
   1. expand the type `Triple[Int, Boolean, String]`, what will be the concrete form?
   2. if we initialize an object with `val t = Triple(1, Single(1.7), Tuple(2.1, Single("some string")))`, 
       1. what will be the types of `SomeTypeA`, `T` and `HELLO` type parameter.
       2. what will be the value of `t.c.b.a` and its type.
   3. why `Triple(1, Single(1.7), Tuple(2.1f, Single("some string")))` throws a compile time error.

5. Implement `concat` method for `List[A]` type. It should join the second list at the end of first list.
   ```
   val list1: List[Int] = Pair(1, Pair(2, Pair(3, End[Int]())))
   val list2: List[Int] = Pair(4, Pair(5, Pair(6, End[Int]())))
   ```
   if inputs are `list1` and `list2`, then result of `concat(list1, list2)` should be `Pair(1, Pair(2, Pair(3, Pair(4, Pair(5, Pair(6, End[Int]()))))))`.